# 🚀 Sentiment Analysis using DistilBERT on CPU

## 📌 Overview
This project builds a **lightweight Sentiment Analysis Model** using **DistilBERT** that:  
✅ **Classifies** text into **Positive** or **Negative**  
✅ **Trains quickly with minimal resources**  
✅ **Deploys as an API using FastAPI**  


## 🛠️ Project Steps
1️⃣ **Install Dependencies**  
2️⃣ **Load a Small Sentiment Analysis Dataset**  
3️⃣ **Preprocess & Tokenize Text**  
4️⃣ **Train a DistilBERT Model on CPU**  
5️⃣ **Deploy as an API using FastAPI**  


## 📌 Step 1: Install Dependencies



In [ ]:
#pip install transformers datasets torch fastapi uvicorn


## 📌 Step 2: Load a Small Sentiment Analysis Dataset

We need a dataset to train the model. **Yelp Review dataset** contains movie reviews labeled as **positive (1)** or **negative (0)**.




#### Importing Required Libraries
The script begins by importing the necessary libraries:

- `torch`: Provides deep learning functionalities.
- `pandas`: Used for handling and processing structured data.
- `datasets`: Loads NLP datasets from Hugging Face.
- `transformers`: Contains pretrained NLP models and training utilities.
- `sklearn.model_selection`: Used to split the dataset into training and test sets.

#### Loading the Yelp Review Dataset
The dataset is loaded using the `datasets` library. It selects **2000 samples** from the Yelp review dataset's training split and converts it into a Pandas DataFrame for easy manipulation.

#### Converting Star Ratings to Binary Sentiment Labels
The dataset originally contains **star ratings from 1 to 5**:
- **3-star reviews** are removed to eliminate neutral sentiments.
- Ratings **4 and 5** are labeled as **positive (1)**.
- Ratings **1 and 2** are labeled as **negative (0)**.
This conversion allows the dataset to be used for binary sentiment classification.

#### Shuffling the Dataset
To ensure randomness, the dataset is shuffled using `df.sample(frac=1, random_state=42)`. The `random_state` parameter ensures that the shuffle order remains consistent across different runs.

#### Displaying Dataset Information
To validate the preprocessing steps:
- `df.head()` prints a preview of the first few records.
- `df["label"].value_counts()` checks the distribution of positive and negative samples to verify balance in the dataset.

This processed dataset is now ready to be used for training a **binary sentiment classification model**.


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the IMDb dataset (only 2000 samples for quick training)
dataset = load_dataset("yelp_review_full", split="train[:2000]")
df = pd.DataFrame(dataset)


# Convert star ratings to binary sentiment labels
df = df[df["label"] != 2]  # Remove 3-star neutral reviews
df["label"] = df["label"].apply(lambda x: 1 if x > 2 else 0)  # 4-5 stars -> Positive (1), 1-2 stars -> Negative (0)

# Shuffle dataset
df = df.sample(frac=1, random_state=42)

# Display dataset info
print(df.head())
print(df["label"].value_counts())  # Check label distribution




     label                                               text
696      1  Named for Pittsburgh greats, Gene Kelly and Bi...
850      1  Love Casbah.  Been going there for years and h...
930      1  This Valentines Day I ordered a pizza for my b...
520      1  We are fans of Pistella Beer Distributors simp...
221      0        Slow service\nBelow average food \nIll pass
label
0    833
1    734
Name: count, dtype: int64


## 📌 Step 3: Data Preprocessing

- Raw text might contain **extra spaces or unnecessary symbols**.  
- **Clean text** to improve training efficiency.


In [ ]:
# 📌 Clean and Preprocess Text
import re

def clean_text(text):
    """Remove extra spaces and unwanted characters."""
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

df["text"] = df["text"].apply(clean_text)
print("✅ Text cleaning completed!")


✅ Text cleaning completed!


## 📌 Step 4: Tokenization

- ML models cannot **directly** process raw text  
- Convert text into **numerical representations (tokens)**  
- **DistilBERT tokenizer** converts words into **token IDs**  


In [ ]:
from transformers import AutoTokenizer

# 📌 Tokenization
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    """Tokenize text with DistilBERT tokenizer."""
    encoding = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    encoding["label"] = example["label"]  # Add labels
    return encoding

# Apply tokenization
df["tokens"] = df.apply(tokenize_function, axis=1)
print("✅ Tokenization completed!")


✅ Tokenization completed!


## 📌 Step 5: Load DistilBERT Model & Train on CPU

- We **use DistilBERT** (lighter version of BERT)  
- Fine-tune the model for **sentiment analysis**    


In [ ]:

# 📌 Prepare Data for Training
# Convert tokenized text into input IDs and attention masks
df["input_ids"] = df["tokens"].apply(lambda x: x["input_ids"])
df["attention_mask"] = df["tokens"].apply(lambda x: x["attention_mask"])

# Drop unnecessary columns
df = df.drop(columns=["tokens"])

# Train-test split (80% train, 20% validation)
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

print(f"✅ Train size: {len(train_dataset)}, Eval size: {len(eval_dataset)}")



✅ Train size: 1253, Eval size: 314


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

# ✅ Load DistilBERT model for Sentiment Analysis (Binary Classification: Positive/Negative)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# ✅ Explicitly use CPU
device = torch.device("cpu")  # Ensures no GPU usage
model.to(device)

print("✅ Model loaded successfully on CPU!")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded successfully on CPU!


In [ ]:
# 📌 Automatically Select GPU if Available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using Device: {device}")


✅ Using Device: cpu


In [ ]:
# 📌 Load DistilBERT Model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)

print("✅ Model loaded successfully!")


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded successfully!


## 📌 **Training the DistilBERT Sentiment Model**
### 🔧 **Key Steps**
✔ Load the **DistilBERT model** with `num_labels=2` (Binary classification)  
✔ Define **training parameters** optimized for CPU  
✔ Use the **Hugging Face Trainer API** to train the model  


In [ ]:
# ✅ Define Training Arguments
training_args = TrainingArguments(
    output_dir="./distilbert_sentiment_cpu",  # Save the model output
    per_device_train_batch_size=4,  # Small batch size for CPU
    num_train_epochs=2,  # Keep it small for quick training
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    learning_rate=3e-5,
    save_total_limit=2,
    no_cuda=True,  # ✅ Force CPU usage
)

# ✅ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Include evaluation dataset to avoid errors
)

# ✅ Start Training
trainer.train()

print("✅ Model Training Complete!")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1590: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shrinidhisudhir (shrinidhisudhir-university-of-houston) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,0.288268
2,0.320100,0.340406


✅ Model Training Complete!


## 📌 **Evaluating Model Performance**
### 🔧 **Key Steps**
✔ Compute **accuracy** on the evaluation dataset  
✔ Use `Trainer.evaluate()` to get loss & metrics  


In [ ]:
# ✅ Evaluate the model
eval_results = trainer.evaluate()
print(f"✅ Evaluation Results: {eval_results}")


✅ Evaluation Results: {'eval_loss': 0.34040603041648865, 'eval_runtime': 146.3712, 'eval_samples_per_second': 2.145, 'eval_steps_per_second': 0.273, 'epoch': 2.0}


## 📌 **Saving the Model for Deployment**
### 🔧 **Key Steps**
✔ Save model & tokenizer for later inference  
✔ Use `save_pretrained()` for easy loading  


In [ ]:
# ✅ Save trained model
model.save_pretrained("./distilbert_sentiment_model")
tokenizer.save_pretrained("./distilbert_sentiment_model")

print("✅ Model and Tokenizer Saved Successfully!")


✅ Model and Tokenizer Saved Successfully!


## 📌 **Testing the Sentiment Analysis Model**
### 🔧 **Key Steps**
✔ Tokenize a **new sentence**  
✔ Use model **to predict sentiment (positive/negative)**  
✔ Convert **logits to probabilities** using softmax  


In [ ]:
import torch
from transformers import AutoTokenizer

# ✅ Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("./distilbert_sentiment_model")
model = AutoModelForSequenceClassification.from_pretrained("./distilbert_sentiment_model")
model.to("cpu")  # ✅ Force CPU usage

# ✅ Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    sentiment = "Positive" if predicted_class == 1 else "Negative"
    return sentiment

# ✅ Test the model with new input
#test_sentence = "I absolutely hated the movie! It was bad."
test = input("Enter a string:")
#predicted_sentiment = predict_sentiment(test_sentence)
predicted_sentiment = predict_sentiment(test)


print(f"✅ Sentiment Prediction: {predicted_sentiment}")


Enter a string:Glad it works!
✅ Sentiment Prediction: Positive


## 📌 **Deploying Sentiment Model as an API**
### 🔧 **Key Steps**
✔ Use **FastAPI** to create an API endpoint  
✔ Accept **user input** & return **sentiment predictions**  
✔ Run API with **Uvicorn**  
